In [1]:
!pip install tensorflow==2.9.0 keras==2.9.0

In [2]:
!pip install keras.utils mtcnn keras_applications

In [3]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-w7i34hkw
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-w7i34hkw
  Resolved https://github.com/rcmalli/keras-vggface.git to commit bee35376e76e35d00aeec503f2f242611a97b38a
  Preparing metadata (setup.py) ... done


In [4]:
!unzip raw.zip

Archive:  raw.zip
replace raw/rasaq.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace raw/mayowa.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras.utils.layer_utils import get_source_inputs
from PIL import Image

In [6]:
# function to extract face from an image

def face_extract(filename, required_size=(224, 224), display=False):
    img = Image.open(filename)
    img = img.convert("RGB")
    pixels = np.asarray(img)

    mtcnn_detector = MTCNN()
    faces = mtcnn_detector.detect_faces(pixels)

    if faces:
      number_of_faces = len(faces)

      if display:
          print(f"Number of faces detected: {number_of_faces}")

      x1, y1, width, height = faces[0]["box"]
      x2, y2 = x1 + width, y1 + height
      face = pixels[y1:y2, x1:x2]
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array= np.asarray(image)
      if display:
          plt.imshow(image)
          plt.axis("off")
          plt.show()

      return face_array

    else:
      return None

In [7]:
from numpy import asarray
import numpy as np
from tensorflow.keras.preprocessing import image

In [8]:
model = VGGFace(model="resnet50",
                  include_top=False,
                  pooling="avg"
                  )

In [9]:
from keras_vggface.utils import decode_predictions

In [10]:
def extract_embeddings(face_array):
  face = face_array.astype("float32")
  face = np.expand_dims(face, axis=0)
  face = preprocess_input(face, version=2)
  embeddings = model.predict(face)
  embeddings = np.squeeze(embeddings, axis=0)

  return embeddings

In [11]:
import os
import json

In [12]:
def process_images(file_directory):
  embeddings_dict ={}
  for filename in os.listdir(file_directory):
    path = os.path.join(file_directory, filename)
    face = face_extract(path)
    if face is not None:
      extracted_embedding = extract_embeddings(face)
      embeddings_dict[filename] = extracted_embedding.tolist()
    else:
      print(f"No face detected in image: {filename}")

  with open("embeddings.json", 'w') as json_file:
    json.dump(embeddings_dict, json_file)

  print("Completed Facial embeddings extraction. Saved to 'embeddings.json'")


In [13]:
process_images("./raw")

1/1 [==============================] - 3s 3s/step


1/1 [==============================] - 1s 605ms/step


1/1 [==============================] - 0s 168ms/step
No face detected in image: adekunle.jpeg
1/1 [==============================] - 0s 182ms/step
Completed Facial embeddings extraction. Saved to 'embeddings.json'


In [14]:
# loading embeddings from the json

def load_embeddings(json_file):
  with open(json_file, "r") as file:
    embeddings_dict = json.load(file)

  for key in embeddings_dict:
    embeddings_dict[key] = np.array(embeddings_dict[key])

  return embeddings_dict

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# similarity match for images

def match_face(new_embedding, embeddings_dict):
  max_similarity = 0
  identity = None
  for filename, stored_embedding in embeddings_dict.items():
    similarity = cosine_similarity(new_embedding.reshape(1, -1), stored_embedding.reshape(1, -1))
    if similarity > max_similarity:
      max_similarity = similarity
      identity = filename
  return identity, max_similarity

In [17]:
# processing new face

def new_face_processing(image_path, embeddings_json= "./embeddings.json"):
  embeddings_dict = load_embeddings(embeddings_json)

  new_face = face_extract(image_path)
  if new_face is not None:
    new_embeddings = extract_embeddings(new_face)

    identity, similarity = match_face(new_embeddings, embeddings_dict)
    if identity:
      print(f"Identified image:{identity} with similarity {similarity}")
    else:
      print("No matching face found")

  else:
    print("No face detected in new image")

In [18]:
new_rasaq = "/content/rasaq_2.jpeg"

In [19]:
new_face_processing(new_rasaq)

1/1 [==============================] - 0s 187ms/step
Identified image:rasaq.jpeg with similarity [[0.74632821]]
